In [1]:
import cv2
import numpy as np
# Hough Transform for lane detection


#function to convert read image 
def read_image(image_path):
    """This function reads any image from a given path"""
    image = cv2.imread(image_path)
    return image

#function to convert BGR image to Gray
def gray_scale(image):
    converted_image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    return converted_image

#function to apply Gaussian filter on Gray Image
def guassin_filter(image, kernel_dimensions):
    filtered_image = cv2.GaussianBlur(image, kernel_dimensions, 0)
    return filtered_image

#function to detect edges using Canny Edge Detector
def canny_edge_detector(image, lower_threshold, upper_threshold):
    canny_detected_edge = cv2.Canny(image, lower_threshold, upper_threshold)
    return canny_detected_edge


def ROI_polygon(image):
    mask = np.zeros_like(image)
    
    if len(image.shape) > 2:
        channel_count = image.shape[2]
        ignore_mask_color = (255,)*channel_count
    
    else:
        ignore_mask_color = 255
    
    rows = image.shape[0]
    columns = image.shape[1]
    
    bottom_left = [0, rows]
    top_left = [columns*0.5, rows*0.5]
    bottom_right = [columns*0.9, rows*0.95]
    top_right = [columns*0.6, rows*0.6]
    
    vertices = np.array([[bottom_left,top_left,top_right,bottom_right]], dtype=np.int32)
    cv2.fillPoly(mask,vertices,ignore_mask_color)
    masked_image = cv2.bitwise_and(image,mask)
    
    return masked_image

#To find just region of interest in the image
def ROI_masking(image):
    height = image.shape[0]
    width = image.shape[1]
    
    image = np.array(image, dtype=np.float32)
    polygon_coordinates = np.array([(0,height),(round(width/2),round(height/2)),(1000,height)])
    mask = np.zeros((height,width))
    masked_image1 = cv2.fillPoly(mask,[polygon_coordinates],255)
    masked_image2 = np.array(masked_image1,dtype=np.float32)
    masked_image3 = cv2.bitwise_and(image,masked_image2)
    
    return masked_image3

#Function to define slope and intercept to find coordinates for left lines
def average_slope_intercept(image,lines):
    left_lines = []
    right_lines = []
    
    for line in lines:
        parameters = np.polyfit((x1,x2),(y1,y2),1)
        slope = parameters[0]
        y_intercept = parameters[1]
        
        if slope < 0:
            left_lines.append((slope,y_intercept))
            
        else:
            right_lines.append((slope,y_intercept))
            
    #Left Line COordinates
    left_average = np.average(left_lines, axis=0)
    right_average = np.average(right_lines, axis=0)
    
    left_line = get_coordinates(image,left_average)
    right_line = get_coordinates(image,right_average)
    
    return np.array([left_line, right_line])

#To get the get coordinates of lines on lanes
def get_coordinates(image, parameters):
    slope, intercept = parameters
    
    if slope < 0:
        y1 = image.shape[0]
        y2 = int(y1*(3/5))
        x1 = np.clip(int(((y1-intercept)/slope)),50,image.shape[1])
        x2 = int((y2-intercept)/slope)
    
    else:
        y1 = image.shape[0]
        y2 = int(y1*(3/5))
        x1 = int((y1-intercept)/slope*0.82)
        x2 = int((y2-intercept)/slope)
    
    return np.array([x1,y1,x2,y2])


#function to draw lines on lanes either side of the road
def draw_lane_lines(image,lines,color,thickness):
    line_image = np.zeros_like(image)
    
    for line in lines:
        x1,y1,x2,y2 = line
        
        if line is not None:
            cv2.line(line_image,(x1,y1),(x2,y2),color,thickness)
            
    return cv2.addWeighted(image,0.8,line_image,1.0,1.0)


#To find hough lines using hough transformation
def hough_transformation(region_of_interest):
    image = cv2.convertScaleAbs(region_of_interest)
    return cv2.HoughLinesP(image, 1, np.pi/180, 50, None, 0, 1000)
    
    
    
    
dims = (5,5)
l_limit = 50
u_limit = 150
line_color = [0,0,255]
line_thickness = 12

image = read_image('road lane.jpg')
gray_image = gray_scale(image)
cv2.imshow('gray image',gray_image)
cv2.waitKey(0)

filtered_image = guassin_filter(image=gray_image, kernel_dimensions=dims)
cv2.imshow('filtered image',np.hstack((gray_image,filtered_image)))
cv2.waitKey(0)

edge_image = canny_edge_detector(image, lower_threshold=l_limit, upper_threshold=u_limit)
cv2.imshow('edge image',np.hstack((filtered_image, edge_image)))
cv2.imshow('edge image',edge_image)
cv2.waitKey(0)



-1